# Notebook Final da Parte Quantitativa

In [32]:
import dask.dataframe as dd
import pandas as pd
import sqlalchemy as db
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pyarrow as pa
import os

In [ ]:
# Ler o arquivo CSV usando dask
df = dd.read_csv('LoteInicial.csv')

# Salvar o DataFrame como parquet
df.to_parquet('Lote_Inicial_completo.parquet')

print("227 .parquets gerados na Pasta: Lote_Inicial_completo.parquet")

In [ ]:
# Ler todos os arquivos Parquet gerados
df = dd.read_parquet('Lote_Inicial_completo.parquet/*.parquet')

# Salvar o DataFrame concatenado em um único arquivo Parquet
df.to_parquet('Lote_Inicial_concatenado.parquet', write_index=False)

print("Todos os arquivos Parquet foram concatenados em Lote_Inicial_concatenado.parquet")

In [29]:
n_arquivos = 30

# Criar uma lista vazia para armazenar os DataFrames
tables = []

# Adicionar o arquivo Parquet 'Lote_Inicial_concatenado.parquet' à lista de tabelas
try:
    lote_inicial_table = pq.read_table('Lote_Inicial_concatenado.parquet')
    tables.append(lote_inicial_table)
    print("Adicionado Lote_Inicial_concatenado.parquet à lista de tabelas.")
except Exception as e:
    print(f"Erro ao ler Lote_Inicial_concatenado.parquet: {e}")


for i in range(n_arquivos):
    csv_file = f"Quanti_202406{i+1:02d}.csv"
    parquet_file = f"Quanti_202406{i+1:02d}.parquet"

    # Verificar se o arquivo CSV existe
    if not os.path.exists(csv_file):
        print(f"Arquivo {csv_file} não encontrado.")
        continue

    # Ler o CSV usando pyarrow
    try:
        table = pv.read_csv(csv_file)
    except Exception as e:
        print(f"Erro ao ler {csv_file}: {e}")
        continue

    # Salvar a tabela em formato Parquet
    try:
        pq.write_table(table, parquet_file)
        print(f"Converteu {csv_file} para {parquet_file}")
    except Exception as e:
        print(f"Erro ao salvar {parquet_file}: {e}")
        continue

    # Adicionar o tabela à lista
    tables.append(table)

# Concatenar todas as tabelas em uma única tabela
if tables:
    # Alinhar os schemas
    schema = tables[0].schema
    aligned_tables = [table.cast(schema) for table in tables]

    concatenated_table = pa.concat_tables(aligned_tables)

    # Converter a tabela concatenada para um DataFrame pandas
    df = concatenated_table.to_pandas()

    # Remover linhas com valores nulos na coluna 'ISIN'
    df = df.dropna(subset=["ISIN"])

    # Converter de volta para uma tabela pyarrow
    concatenated_table = pa.Table.from_pandas(df)


    # Salvar a tabela concatenada em formato Parquet
    try:
        pq.write_table(concatenated_table, "dados_concatenados.parquet")
        print("Todos os arquivos foram convertidos e concatenados em Parquet.")
    except Exception as e:
        print(f"Erro ao salvar dados_concatenados.parquet: {e}")
else:
    print("Nenhuma tabela foi carregada.")

Erro ao salvar Quanti_20240601.parquet: [WinError 5] Failed to open local file 'Quanti_20240601.parquet'. Detail: [Windows error 5] Acesso negado.

Converted Quanti_20240602.csv to Quanti_20240602.parquet
Converted Quanti_20240603.csv to Quanti_20240603.parquet
Converted Quanti_20240604.csv to Quanti_20240604.parquet
Converted Quanti_20240605.csv to Quanti_20240605.parquet
Converted Quanti_20240606.csv to Quanti_20240606.parquet
Converted Quanti_20240607.csv to Quanti_20240607.parquet
Converted Quanti_20240608.csv to Quanti_20240608.parquet
Converted Quanti_20240609.csv to Quanti_20240609.parquet
Converted Quanti_20240610.csv to Quanti_20240610.parquet
Converted Quanti_20240611.csv to Quanti_20240611.parquet
Converted Quanti_20240612.csv to Quanti_20240612.parquet
Converted Quanti_20240613.csv to Quanti_20240613.parquet
Converted Quanti_20240614.csv to Quanti_20240614.parquet
Converted Quanti_20240615.csv to Quanti_20240615.parquet
Converted Quanti_20240616.csv to Quanti_20240616.parqu

In [30]:
concatenated_table.columns

 [
   [
     "LU0529382367",
     "LU0214705203",
     "LU0511383332",
     "LU0511383688",
     "IE00B5N0GF80",
     ...
     "IE0009AP1390",
     "IE000FL2SDR4",
     "IE000IM4K4K2",
     "FR0014008R11",
     "IE000BF4EX42"
   ]
 ],
 [
   [
     102514737.14,
     48051400,
     2778680208.04,
     2778680208.04,
     2167900562.2,
     ...
     2165813019.75,
     273227467.59,
     3760255.3,
     21307656.24,
     2610013667.22
   ]
 ],
 [
   [
     2024-05-31,
     2024-05-31,
     2024-05-31,
     2024-05-31,
     2024-05-31,
     ...
     2024-06-28,
     2024-06-27,
     2024-06-28,
     2024-06-27,
     2024-06-28
   ]
 ],
 [
   [
     56805261.72,
     7848120.99,
     29996701.88,
     15013115.88,
     8218169008.51,
     ...
     23642.93,
     109265666.74,
     3760255.3,
     84619.54,
     276979272.41
   ]
 ],
 [
   [
     2024-05-30,
     2024-05-31,
     2024-05-31,
     2024-05-31,
     2024-05-31,
     ...
     2024-06-28,
     2024-06-28,
     2024-06-28,
     2

In [8]:
#Prioritário
df1 = concatenated_table[["ISIN","Valuation NAV","Valuation NAV Date"]].sort_values(by=["ISIN","Valuation NAV Date"])
df2 = concatenated_table[["ISIN","FE Bid",'FE Bid Date']].sort_values(by=["ISIN","FE Bid Date"])
df3 = concatenated_table[["ISIN","FE Generic NAV","FE Generic NAV Date"]].sort_values(by=["ISIN","FE Generic NAV Date"])
df4 = concatenated_table[["ISIN","FE NAV","FE NAV Date"]].sort_values(by=["ISIN","FE NAV Date"])
df5 = concatenated_table[["ISIN","Transaction NAV","Transaction NAV Date"]].sort_values(by=["ISIN","Transaction NAV Date"])
fields = ["Valuation NAV","FE BID","FE Generic NAV","FE NAV","Transaction NAV"]

In [9]:
#Extras
df6 = concatenated_table[["ISIN","AuM Fund","AuM Fund Date"]].sort_values(by=["ISIN","AuM Fund Date"]),
df7 = concatenated_table[["ISIN","AuM Share Class","AuM Share Class Date"]].sort_values(by=["ISIN","AuM Share Class Date"]),
df8 = concatenated_table[["ISIN","Ask NAV","Ask NAV Date"]].sort_values(by=["ISIN","Ask NAV Date"]),
df9 = concatenated_table[["ISIN","Bid NAV","Bid NAV Date"]].sort_values(by=["ISIN","Bid NAV Date"]),
df10 = concatenated_table[["ISIN","NoS Fund","NoS Fund Date"]].sort_values(by=["ISIN","NoS Fund Date"]),
df11 = concatenated_table[["ISIN","NoS Share Class","NoS Share Class Date"]].sort_values(by=["ISIN","NoS Share Class Date"]),
df12 = concatenated_table[["ISIN","FE Offer","FE Offer Date"]].sort_values(by=["ISIN","FE Offer Date"]),
df13 = concatenated_table[["ISIN","FE AUM Fund","FE AUM Fund Date"]].sort_values(by=["ISIN","FE AUM Fund Date"]),
df14 = concatenated_table[["ISIN","FE AUM Share Class","FE AUM Share Class Date"]].sort_values(by=["ISIN","FE AUM Share Class Date"]),
df15 = concatenated_table[["ISIN","FE NOS Fund","FE NOS Fund Date"]].sort_values(by=["ISIN","FE NOS Fund Date"]),
df16 = concatenated_table[["ISIN","FE NOS Share Class","FE NOS Share Class Date"]].sort_values(by=["ISIN","FE NOS Share Class Date"]),

fields = ["Valuation NAV","FE BID","FE Generic NAV","FE NAV","Transaction NAV","AuM Fund","AuM Share Class","Ask NAV",
"Bid NAV","NoS Fund","NoS Share Class","FE Offer","FE AUM Fund","FE AUM Share Class","FE NOS Fund","FE NOS Share Class"],
ldf = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16]

In [10]:
isins = concatenated_table["ISIN"].unique()

In [11]:
ldf = [df1,df2,df3,df4,df5]

In [12]:
col_names = ["ISIN","data_valor_antigo","data_valor_novo","campo","valor antigo","valor novo" ]
new_df = pd.DataFrame(columns=col_names)

In [13]:
# Inicializa a lista de DataFrames e outras variáveis
campo = 0
new_df_list = []  # Lista para armazenar os DataFrames temporários

for df in ldf:
    for isin in isins:
        actual = df[df["ISIN"] == isin]

        for j in range(len(actual) - 1):
            row = actual.iloc[j]
            next_row = actual.iloc[j + 1]
            if row[1] != next_row[1]:
                new_df_list.append({
                    "ISIN": row["ISIN"],
                    "campo": fields[campo],
                    "data_valor_antigo": row[2],
                    "data_valor_novo": next_row[2],
                    "valor_antigo": row[1],
                    "valor_novo": next_row[1]
                })

    campo += 1

# Concatena todos os dicionários em um único DataFrame
new_df = pd.DataFrame(new_df_list)


IndexError: boolean index did not match indexed array along dimension 0; dimension is 15558 but corresponding boolean dimension is 13722

In [ ]:
# Inicializa a lista de DataFrames e outras variáveis
# Iterar por cada grupo de ISIN
for isin, group in df1.groupby('ISIN'):
    # Ordenar por data dentro de cada grupo
    group = group.sort_values(by=df1.columns[2])

    # Verificar se há mudanças de valor entre linhas consecutivas
    for i in range(1, len(group)):
        valor_antigo = group.iloc[i - 1][df1.columns[1]]
        valor_novo = group.iloc[i][df1.columns[1]]
        data_antiga = group.iloc[i - 1][df1.columns[2]]
        data_nova = group.iloc[i][df1.columns[2]]

        if valor_antigo != valor_novo:  # Detectar mudança no valor
            new_df = new_df.append({
                "ISIN": isin,
                "data_valor_antigo": data_antiga,
                "data_valor_novo": data_nova,
                "valor_antigo": valor_antigo,
                "valor_novo": valor_novo
            }, ignore_index=True)


new_df

In [ ]:
# Concatenar os DataFrames em um único, mantendo as colunas relevantes
dados = pd.concat([df1, df2, df3, df4, df5])

# Ordenar pelos ISIN e pela data correspondente
dados.sort_values(by=["ISIN", "Valuation NAV Date"], inplace=True)

# Criar um DataFrame deslocado para comparações
dados['valor_antigo'] = dados.groupby('ISIN')['Valuation NAV'].shift(1)
dados['data_valor_antigo'] = dados.groupby('ISIN')['Valuation NAV Date'].shift(1)

# Filtrar as linhas onde o valor mudou
mudancas = dados[dados['Valuation NAV'] != dados['valor_antigo']].copy()

# Selecionar as colunas relevantes
mudancas_df = mudancas[['ISIN', 'data_valor_antigo', 'Valuation NAV Date', 'valor_antigo', 'Valuation NAV']]
mudancas_df.columns = ['ISIN', 'data_valor_antigo', 'data_valor_novo', 'valor_antigo', 'valor_novo']

In [ ]:
mudancas_df

In [ ]:
server = 'localhost'
database = 'master'
db.create_engine(f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server;Trusted_Connection=yes;')

In [ ]:
engine = db.create_engine(f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server;Trusted_Connection=yes;')
df.to_sql('Quali', con=engine, index=False, if_exists='replace')